In [1]:
import json
import csv
import pandas as pd
import os

# Go up one dir

if not os.path.exists("ads/"):
    os.chdir("..")

In [2]:
import warnings
warnings.filterwarnings("ignore")
        
def get_tagging(info, database):
    # info is a tuple (video_url, site)
    
    url, site = info
    
    url_db = database[database["preroll_ad_video_url"]==url]
    if len(url_db) != 0:
        return url_db.iloc[0]["tag"], url_db.iloc[0]["is_scam"]
    
    site_db = database[database["preroll_ad_site"]==site]
    if len(site_db) != 0:
        return site_db.iloc[0]["tag"], site_db.iloc[0]["is_scam"]
    
    return None, None


# build the database
def build_database():
    tag_header = ["preroll_ad_video_url", "preroll_ad_site", "preroll_ad_advertiser",
                  "preroll_ad_location", "id", "tag", "is_scam"]

    run_ids = ["20230518_1957", "20230526_1750", "20230527_2230",
               "20230530_1949", "20230529_2213", "20230601_1751",
               "20230606_0332", "20230606_1947"]

    db = pd.read_excel("ads/preroll_ads_tagging.xlsx", sheet_name = run_ids)
    dfs = []

    for run_id in run_ids:
        df = db.get(run_id)
        df = df[df['preroll_ad_video_url'].notna()]
        df['run_id'] = run_id
        dfs.append(df)

    database = pd.concat(dfs, ignore_index=True)
    return database

In [3]:
database = build_database()
database

,preroll_ad_video_url,preroll_ad_site,preroll_ad_advertiser,preroll_ad_location,tag,is_scam,Notes,run_id,df_index,id
0,https://www.youtube.com/watch?v=LCTqyxEUmHU,parasitesnomore.com,PureHealth Research LLC,United States,Health Products,1,NaN,20230518_1957,NaN,NaN
1,https://www.youtube.com/watch?v=DcD1tOMiIvk,pacificrimathletics.com/online-reg,Pacific Rim Athletics,United States,Fitness,1,"athletic training, 10k for a short course",20230518_1957,NaN,NaN
2,https://www.youtube.com/watch?v=jR-gYFNLVxs,click.doodly.com,Voomly LLC,United States,Software Services,0,sketchy hidden subscription,20230518_1957,NaN,NaN
3,https://www.youtube.com/watch?v=PahO1czqaqM,go.masterclass-piano.com,Creators Secrets Inc.,United States,Educational Services,1,NaN,20230518_1957,NaN,NaN
4,https://www.youtube.com/watch?v=kuQIm72_H3o,cf.spybriefing.com,NaN,NaN,Fitness,1,"self-defense training, very sketchy",20230518_1957,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1223,https://www.youtube.com/watch?v=gpUNmJgM7IQ,jaychrismentor.com,NaN,NaN,Fitness,1,NaN,20230606_1947,NaN,1964.0
1224,https://www.youtube.com/watch?v=FT5KAoUOEKE,joinhoney.com/youtube,"PayPal, Inc.",United States,Software Services,0,NaN,20230606_1947,NaN,1977.0
1225,https://www.youtube.com/watch?v=0O6ys4Xm3TI,warthunder.com,Gaijin Distribution Korlátolt Felelősségű Társ...,Hungary,Games,0,NaN,20230606_1947,NaN,1977.0
1226,https://www.youtube.com/watch?v=uq6jd2D8HV0,pacorabanne.com/invictus-elixir,NaN,NaN,Beauty,0,NaN,20230606_1947,NaN,1990.0


In [13]:
id = "20230529_2213"

run_ids = ["20230518_1957", "20230526_1750", "20230527_2230",
               "20230530_1949", "20230529_2213", "20230601_1751",
               "20230606_0332", "20230606_1947"]

ad_json_file = open(f"ads/ads_{id}.json", "r", encoding="utf-8")

for line in ad_json_file:
    json_data = json.loads(line)

    if json_data['preroll_ad_video_url'] and json_data['preroll_ad2_video_url']:
        if json_data['preroll_ad_video_url'] != json_data['video_url'] and json_data['preroll_ad2_video_url'] != json_data['video_url']:

            video_url, site = json_data["preroll_ad_video_url"], json_data["preroll_ad_site"]
            video_url2, site2 = json_data["preroll_ad2_video_url"], json_data["preroll_ad2_site"]
            tag, is_scam = get_tagging((video_url, site), database)
            tag2, is_scam2 = get_tagging((video_url2, site2), database)
            if json_data['preroll_ad_reasons']:
                print(json_data['preroll_ad_reasons'])
            if json_data['preroll_ad2_reasons']:
                print(json_data['preroll_ad2_reasons'])

['Your gender', 'The time of day or your general location (like your country or city)']
['Your gender', "The video you're watching", 'The time of day or your general location (like your country or city)']
['Your age', "The video you're watching", 'The time of day or your general location (like your country or city)']
['Your age', 'The time of day or your general location (like your country or city)', "The video you're watching"]
['Your gender', 'Your age', 'The time of day or your general location (like your country or city)', "Google's estimation of your interests, based on your activity while you were signed in to Google"]
['The time of day or your general location (like your country or city)']
['Your age', 'The time of day or your general location (like your country or city)', "Google's estimation of your interests, based on your activity while you were signed in to Google"]
['Your age', "The video you're watching", 'The time of day or your general location (like your country or cit